# TelcoGenie 1.0

## 1. Introduction
### 1.1 Business Domain
    The telecom industry is at the forefront of digital transformation, driven by technological advancements and the demand for enhanced customer experience. Generative AI offers significant potential to redefine customer service, automating repetitive queries while providing personalized, engaging interactions.

### 1.2 Overview
    TelcoGenie 1.0 is a conversational Generative AI chatbot designed for telecom companies. This product aims to automate and streamline customer support for mobile subscription-related queries. The chatbot caters to prepaid and postpaid customers by handling inquiries about balance, billing, pricing plans, and recommending suitable plans.

### 1.3 Scope
    Included Features:
    Balance inquiries for prepaid customers.
    Billing information for postpaid customers.
    Pricing plan details for both prepaid and postpaid customers.
    Plan recommendations for potential and existing customers.

    Excluded Features:
    Queries unrelated to mobile subscriptions.
    Other telecom services (e.g., broadband, TV).

    Target Users:
    Mobile subscribers (prepaid/postpaid).
    Potential customers looking for a mobile subscription.

### 1.4 Objectives
    Automate basic customer support interactions.
    Reduce reliance on human agents for routine queries.
    Deliver personalized recommendations for user retention and acquisition.



## Import Libraries

In [1]:
# Import the libraries
import os, json, ast
import openai
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [2]:
# Read the OpenAI API key
openai.api_key = open("C:/Users/User/Desktop/UpgradAIML/GENAI/chavi.txt", "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

## Stage 1: Initialize Conversation & Read Data

### 1.1 Start Conversation (init_conversation)

In [71]:
def init_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    
    
    system_message = f"""
    You are an intelligent telecom customer service specialist and your goal is to help users with their queries.
    You need to ask relevant questions and understand the users intent by analysing the user's responses.
    You specialize in dealing with mobile customers. 
    The type of user queries that you can help with are:
    1. Prepaid mobile balance enquiry
    2. Postpaid mobile billing enquiry
    3. Get current Plan details
    4. Recommend top plans based on user's requirement
    
    First you must identify the user's intent. 
    Then you should make use of the supplied tools to assist the user and answer their queries.
    
    {delimiter}
    While using the supplied tools, ensure that you are passing the correct parameter values as per the specifications.
    {delimiter}
    
    Follow the chain-of-thoughts below: \n
    {delimiter}
    Thought 1: Ask a question to understand the user's intent. \n
    If their intent is unclear. Ask followup questions to understand their intent.
    
    You need to first be clear about the user's intent. 
    If the intent is one of the following then you proceed to next thought:
    1. Prepaid mobile balance enquiry
    2. Postpaid mobile billing enquiry
    3. Get current Plan details
    4. New mobile connection or change plan
    
    If the intent is different from any of the above then you tell the user that you will redirect their query to a 
    human assistant and mention that they will soon be connected to a human assistant to help them with their query.
    
    In case you do not have an appropriate tool to be called then continue with the conversation.

    {delimiter}
    Thought 2: Lets say the users intent is one of the below:
    1. Prepaid mobile balance enquiry
    2. Postpaid mobile billing enquiry
    3. Get current plan details
    4. New mobile connection
    5. Change plan
    Check the tools supplied and call the appropriate tool based on the intent.
        
    {delimiter}
    
    {delimiter}
    Thought 3: 
    Do not assume or manipulate the values for any of the parameters input to the tool. 
    You are supposed to get complete understanding of users requirement on all aspects.
    You cannot assume user preferences on any of the aspects like International calls or OTT.
    If the user preference is not clear to you, then ask more questions, do not assume or manipulate the parameter values.
    You must ask question with a sound logic as opposed to directly citing the key you want to understand value for.
    You will be heavily penalised if you manipulate any parameter values without understanding the users requirement.
            
    {delimiter}

    {delimiter}
    Thought 4: Check if you have populated the input parameter values correctly if you are calling a tool.
    For calling the functions get_billing_info or get_balance_info, you must supply value of "Mobile Number" in the input. 
    You will be heavily penalised if you do not follow this instruction.
    If you are not confident about any of the values, ask clarifying questions.
    {delimiter}
    
    {delimiter}
    Thought 5: While asking for the Mobile Number, it is better to specify the expected length
    For example : Please provide your 10 digit mobile number.
    {delimiter}
    
    {delimiter}
    Thought 6: When the user asks for a new mobile connection or a plan change, ask user preferences using questions framed with sound logic.
    You should not ask multiple questions in one go.
    Below is a bad example of asking questions as here multiple questions are asked with direct refernce to low/medium/high values.
    'To help you with a new mobile connection, I need to understand your usage preferences. This will help me recommend the best plans for you. Could you please let me know:\n\n1. Your data usage preference (low, medium, high)?\n2. Your voice usage preference (low, medium, high)?\n3. Your SMS usage preference (low, medium, high)?\n4. Your use of international calling (low, medium, high)?\n5. Your use of international roaming (low, medium, high)?\n6. Your interest in OTT services (low, medium, high)?\n7. Your monthly budget for the mobile plan?'
    A better way to frame the question would be as below :
    'To help you with a new mobile connection, I need to understand your usage preferences. This will help me recommend the best plans for you. Could you please let me know if you use a lot of mobile data on a daily basis?'
    Once the user has answered you can move to the next logical question.
    {delimiter}
    
    {delimiter}
    Thought 7: The functions get_billing_info and get_balance_info will provide a response in a json format. 
    You need to use the details to answer the specific query from the user.
    If the answer to user's query is not available in the response provided by the tools then advise the user that you will connect him/her to a human assistant for further help.
    {delimiter}
    
    {delimiter}
    Thought 8: The function get_plan_recommendations will output a string containing a list of dictionaries.
    Each dictionary in the list represents the details of one plan.
    You need to use these details and recommend the plans to the users in the increasing order of monthly budget.
    You recommend each plan to the user as a sales expert highlighting its key features so that the user will be interested to enroll for the plan. 
    If the function does not return any plan in the response then advise the user that you will connect him/her to a human assistant for further help.
    User may ask more queries about the plans, answer them based on the recommended plans data you have. If the answer to user's query is not available then advise the user that you will connect him/her to a human assistant for further help.
    {delimiter}
    
    
    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    # conversation = system_message
    return conversation

In [72]:
debug_conversation = init_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n    You are an intelligent telecom customer service specialist and your goal is to help users with their queries.\n    You need to ask relevant questions and understand the users intent by analysing the user\'s responses.\n    You specialize in dealing with mobile customers. \n    The type of user queries that you can help with are:\n    1. Prepaid mobile balance enquiry\n    2. Postpaid mobile billing enquiry\n    3. Get current Plan details\n    4. Recommend top plans based on user\'s requirement\n    \n    First you must identify the user\'s intent. \n    Then you should make use of the supplied tools to assist the user and answer their queries.\n    \n    ####\n    While using the supplied tools, ensure that you are passing the correct parameter values as per the specifications.\n    ####\n    \n    Follow the chain-of-thoughts below: \n\n    ####\n    Thought 1: Ask a question to understand the user\'s intent. \n\n    If their intent is unclear. A

In [73]:
system_message = init_conversation()
print(system_message[0]["content"])


    You are an intelligent telecom customer service specialist and your goal is to help users with their queries.
    You need to ask relevant questions and understand the users intent by analysing the user's responses.
    You specialize in dealing with mobile customers. 
    The type of user queries that you can help with are:
    1. Prepaid mobile balance enquiry
    2. Postpaid mobile billing enquiry
    3. Get current Plan details
    4. Recommend top plans based on user's requirement
    
    First you must identify the user's intent. 
    Then you should make use of the supplied tools to assist the user and answer their queries.
    
    ####
    While using the supplied tools, ensure that you are passing the correct parameter values as per the specifications.
    ####
    
    Follow the chain-of-thoughts below: 

    ####
    Thought 1: Ask a question to understand the user's intent. 

    If their intent is unclear. Ask followup questions to understand their intent.
    
   

### 1.2. Read Data

In [74]:
import pandas as pd

In [75]:
plan_data = pd.read_csv("plan_data.csv")

In [76]:
plan_data.head()

,Name,Description,Subscription Type,Bundled Plan Indicator,Minimum Recharge/Monthly Recurring Amount,Free Voice Minutes,Free SMS,Free Data (GB),Validity (Days),Data Speed (Mbps),Value Added Services,Bundled OTT Content
0,Basic Saver,A budget-friendly prepaid plan offering essent...,Prepaid,No,99,100,50,1,28,4,NaN,None
1,Daily Data Delight,A prepaid plan for data enthusiasts; Includes ...,Prepaid,No,249,500,100,56,28,10,NaN,None
2,Streaming Starter,A bundled prepaid plan with 1-month Disney Hot...,Prepaid,Yes,399,300,100,84,28,10,NaN,Disney Hotstar
3,Value Voice,Affordable prepaid plan for voice-focused user...,Prepaid,No,149,1000,50,0.5,28,4,NaN,None
4,Max Connect Pro,A premium bundled prepaid plan offering Netfli...,Prepaid,Yes,999,1000,500,360,90,25,International Roaming; ISD Calling,Netflix


In [77]:
subscriber_data = pd.read_csv("telecom_subscribers_data.csv")

In [78]:
subscriber_data.head()

,Mobile Number,Customer ID,First Name,Last Name,Subscription Type,Subscription Status,Price Plan,Activation Date,Cancellation Date,Total Prepaid Balance,Prepaid Balance Used,Prepaid Balance Remaining,Last Recharge Amount,Last Recharge Date,Last Billed Amount,Last Bill Payment Amount,Last Bill Payment Date,Pending Amount Due,Bill Due Date
0,8924736643,60954206,Nisha,Mehta,Prepaid,Active,Super Saver Plus,2015-07-06,NaN,664.0,157.0,507.0,664.0,2020-02-02,NaN,NaN,NaN,NaN,NaN
1,7446370198,56195994,Anika,Patel,Prepaid,Cancelled,Daily Data Delight,2019-01-12,2023-12-23,809.0,344.0,465.0,809.0,2019-10-21,NaN,NaN,NaN,NaN,NaN
2,9037977779,88330030,Arjun,Kulkarni,Prepaid,Active,Traveler’s Delight,2023-05-20,NaN,438.0,148.0,290.0,438.0,2024-06-09,NaN,NaN,NaN,NaN,NaN
3,8523964407,32464071,Rahul,Singh,Prepaid,Active,Smart Flex,2014-03-18,NaN,227.0,129.0,98.0,227.0,2017-02-10,NaN,NaN,NaN,NaN,NaN
4,7841376909,96075134,Pooja,Patel,Prepaid,Active,Super Saver Plus,2014-08-17,NaN,202.0,134.0,68.0,202.0,2015-01-23,NaN,NaN,NaN,NaN,NaN


## Stage 2: Tools Schema & Function Definitions

### 2.1 Tools Schema

In [124]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_balance_info",
            "description": "Get the prepaid balance information for a mobile subscriber. Call this whenever you need to know prepaid balance related information, for example when a customer asks 'I have an issue with my prepaid number' or 'I want to know my mobile balance'",
            "parameters": {
                "type": "object",
                "properties": {
                    "Mobile_Number": {
                        "type": "string",
                        "description": "The customer's Mobile Number.",
                    }
                },
                "required": ["Mobile_Number"],
                "additionalProperties": False,
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_billing_info",
            "description": "Get the postpaid billing information for a mobile subscriber. Call this whenever you need to know postpaid billing related information, for example when a customer asks 'I have a billing issue' or 'I want to know my bill due date'",
            "parameters": {
                "type": "object",
                "properties": {
                    "Mobile_Number": {
                        "type": "string",
                        "description": "The customer's Mobile Number.",
                    }
                },
                "required": ["Mobile_Number"],
                "additionalProperties": False,
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_customer_details",
            "description": "Get the customer details for a mobile subscriber including the plan details. Call this whenever the user asks for subscription details or plan details or activation date",
            "parameters": {
                "type": "object",
                "properties": {
                    "Mobile_Number": {
                        "type": "string",
                        "description": "The customer's Mobile Number.",
                    }
                },
                "required": ["Mobile_Number"],
                "additionalProperties": False,
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_plan_recommendations",
            "description": "Get top plan recommendations for a mobile subscriber. The subscriber may be a new customer who wants a new connection or an existing subscriber who wants to change the plan. Call this whenever you need to provide plan recommendations to a user, for example when a user asks 'I want a new mobile connection' or 'I want to change my mobile plan'",
            "parameters": {
                "type": "object",
                "properties": {
                    "Data_Usage": {
                        "type": "string",
                        "enum": ["low","medium","high"],
                        "description": "The user's preference on Data Usage (low/medium/high).",
                    },
                    "Voice_Usage": {
                        "type": "string",
                        "enum": ["low","medium","high"],
                        "description": "The user's preference on Voice Usage (low/medium/high).",
                    },
                    "SMS_Usage": {
                        "type": "string",
                        "enum": ["low","medium","high"],
                        "description": "The user's preference on SMS Usage (low/medium/high).",
                    },
                    "International_Calling": {
                        "type": "string",
                        "enum": ["low","medium","high"],
                        "description": "The user's preference on use of International Calling feature (low/medium/high).",
                    },
                    "International_Roaming": {
                        "type": "string",
                        "enum": ["low","medium","high"],
                        "description": "The user's preference on use of International Roaming feature (low/medium/high).",
                    },
                    "OTT_Usage": {
                        "type": "string",
                        "enum": ["low","medium","high"],
                        "description": "The user's preference on OTT usage (low/medium/high).",
                    },
                    "Monthly_Budget": {
                        "type": "integer",
                        "description": "The user's monthly budget.",
                    }
                },
                "required": ["Data_Usage", "Voice_Usage", "SMS_Usage", "International_Calling", "International_Roaming", "OTT_Usage", "Monthly_Budget"],
                "additionalProperties": False,
            }
        }
    }
]

### 2.2 Function Definition: get_customer_details 

In [147]:
import pandas as pd
import json

def get_customer_details(mobile_number, subscription_type = ''):
    
    df = pd.read_csv("telecom_subscribers_data.csv")
        
    # Check if both inputs are missing
    if not mobile_number:
        return json.dumps({"error": "Input missing please provide Mobile Number"})
    
    # Filter the DataFrame based on provided input
    if subscription_type:
        result = df[(df['Mobile Number'] == int(mobile_number)) & (df['Subscription Type'] == subscription_type)]
    else:
        result = df[df['Mobile Number'] == int(mobile_number)]
    
    # Check if any record is found
    if result.empty:
        print("No record found")
        return json.dumps({"error": "No record found for the provided input"})
    
    # Get the first matched record
    record = result.iloc[0]
    
    #json_data = result.to_dict(orient="records")
    #print(json_data)
    
    
    # Create the dictionary based on Subscription Type
    if record['Subscription Type'] == 'Prepaid':
        data = {
            "Mobile Number": int(record['Mobile Number']),
            "Customer ID": int(record['Customer ID']),
            "First Name": record['First Name'],
            "Last Name": record['Last Name'],
            "Subscription Type": record['Subscription Type'],
            "Subscription Status": record['Subscription Status'],
            "Price Plan": record['Price Plan'],
            "Activation Date": record['Activation Date'],
            "Cancellation Date": record['Cancellation Date'],
            "Total Prepaid Balance": float(record['Total Prepaid Balance']),
            "Prepaid Balance Used": float(record['Prepaid Balance Used']),
            "Prepaid Balance Remaining": float(record['Prepaid Balance Remaining']),
            "Last Recharge Amount": float(record['Last Recharge Amount']),
            "Last Recharge Date": record['Last Recharge Date']
        }
    elif record['Subscription Type'] == 'Postpaid':
        data = {
            "Mobile Number": int(record['Mobile Number']),
            "Customer ID": int(record['Customer ID']),
            "First Name": record['First Name'],
            "Last Name": record['Last Name'],
            "Subscription Type": record['Subscription Type'],
            "Subscription Status": record['Subscription Status'],
            "Price Plan": record['Price Plan'],
            "Activation Date": record['Activation Date'],
            "Cancellation Date": record['Cancellation Date'],
            "Last Billed Amount": float(record['Last Billed Amount']),
            "Last Bill Payment Amount": float(record['Last Bill Payment Amount']),
            "Last Bill Payment Date": record['Last Bill Payment Date'],
            "Pending Amount Due": float(record['Pending Amount Due']),
            "Bill Due Date": record['Bill Due Date']
        }
    else:
        return json.dumps({"error": "Invalid Subscription Type"})
    
    # Convert to native Python types
    # data = {key: (value.item() if hasattr(value, 'item') else value) for key, value in data.items()}
    
    # Convert the dictionary to a JSON object and return
    return json.dumps(data)
    
# Example usage
# Assuming `df` is your pandas DataFrame containing the data
# result = extract_customer_data(df, Mobile_Number="1234567890")
# print(result)


### 2.3. Function Definition : get_billing_info

In [148]:
def get_billing_info(Mobile_Number = ''):
    response = get_customer_details(Mobile_Number, "Postpaid")
    return response

### 2.4. Function Definition : get_balance_info

In [149]:
def get_balance_info(Mobile_Number = ''):
    response = get_customer_details(Mobile_Number, "Prepaid")
    return response

### 2.5. Function Definition : plan_map

In [150]:
def plan_map(plan_description):
    delimiter = "#####"
    
    plan_spec = {
        "Data_Usage":"(Free Data allowed in GB)",
        "Voice_Usage":"(Free Voice allowed in minutes)",
        "SMS_Usage":"(Free SMS allowed)",
        "International_Calling":"(Is international calling included as a value added service)",
        "International_Roaming":"(Is international roaming included as a value added service)",
        "OTT_Usage": "(OTT platforms included in the plan)"
    }

    values = {'low','medium','high'}

    prompt=f"""
    You are a mobile plans Specifications Classifier whose job is to extract the key features of mobile plans and classify them as per their requirements.
    To analyze each mobile plan, perform the following steps:
    Step 1: Extract the plan's primary features from the description {plan_description}
    Step 2: Store the extracted features in {plan_spec} \
    Step 3: Classify each of the items in {plan_spec} into {values} based on the following rules: \
    {delimiter}
    Data_Usage:
    - low: <<< If free data in the plan is less than 25 GB >>> , \n
    - medium: <<< If free data in the plan is >= 25 GB and less than 100 GB >>> , \n
    - high: <<< If the free data in the plan is more than 100 GB or unlimited >>> , \n

    Voice_Usage:
    - low: <<< If free voice minutes is less than 300 >>> , \n
    - medium: <<< If free voice minutes is >= 300 and less than 1000 >>> , \n
    - high: <<< If free voice minutes is >=1000 or unlimited >>> \n

    SMS_Usage:
    - low: <<< If free SMS is up to 100 >>> , \n
    - medium: <<< if free SMS is between 100 and 1000 >>> , \n
    - high: <<< if free SMS is greater than 1000 or unlimited >>> \n

    International_Calling:
    - low: <<< If the value added service does not have International Calling >>> , \n
    - high: <<< If the value added service has International Calling >>> \n

    International_Roaming:
    - low: <<< If the value added service does not have International Roaming >>> , \n
    - high: <<< If the value added service has International Roaming >>> \n
    
    OTT_Usage:
    - low: <<< If there is no OTT offered in the plan >>> , \n
    - medium: <<< if there is only one OTT service offered in the plan >>> , \n
    - high: <<< if there are more than one OTT service offered in the plan >>> \n
    {delimiter}

    
    {delimiter}
    ### Strictly don't keep any other text in the values of the JSON dictionary other than low or medium or high ###
    ### The output must only have a JSON dictionary and no other characters before or after it. You will be heavily penalized if you do not follow this ###
    
    """
    input = f"""Follow the above instructions step-by-step and output the dictionary in JSON format {plan_spec} for the following plan {plan_description}. Return only the json in output and no extra characters"""
    #see that we are using the Completion endpoint and not the Chatcompletion endpoint
    messages=[{"role": "system", "content":prompt },{"role": "user","content":input}]

    # JSON return type specified
    MODEL = 'gpt-4o'
    chat_completion_json = openai.chat.completions.create(
            model=MODEL,
            response_format = { "type": "json_object"},
            messages=messages)

    #print(chat_completion_json.choices[0].message.content)
    #response = chat_completion_json.choices[0].message.content
    response = json.loads(chat_completion_json.choices[0].message.content)
    return response

In [129]:
plan_data.iloc[0]['Description']

'A budget-friendly prepaid plan offering essential features for light users; Includes 100 free voice minutes; 50 SMS; and 1 GB data for 28 days at a speed of 4 Mbps; Ideal for minimal usage.'

In [130]:
plan_map(plan_data.iloc[0]['Description'])

{'Data_Usage': 'low',
 'Voice_Usage': 'low',
 'SMS_Usage': 'low',
 'International_Calling': 'low',
 'International_Roaming': 'low',
 'OTT_Usage': 'low'}

In [86]:
plan_data.iloc[4]['Description']

'A premium bundled prepaid plan offering Netflix for 3 months; 1000 voice minutes; 500 SMS; and 4 GB data daily at speeds up to 25 Mbps with a 90-day validity.'

In [87]:
plan_map(plan_data.iloc[4]['Description'])

{'Data_Usage': 'high',
 'Voice_Usage': 'high',
 'SMS_Usage': 'medium',
 'International_Calling': 'low',
 'International_Roaming': 'low',
 'OTT_Usage': 'medium'}

In [88]:
##Run this code once to extract plan info in the form of a dictionary
plan_df= pd.read_csv('plan_data.csv')

## Create a new column "plan_feature" that contains the dictionary of the plan features
plan_df['plan_feature'] = plan_df['Description'].apply(lambda x: plan_map(x))

In [89]:
plan_df.head(5)

,Name,Description,Subscription Type,Bundled Plan Indicator,Minimum Recharge/Monthly Recurring Amount,Free Voice Minutes,Free SMS,Free Data (GB),Validity (Days),Data Speed (Mbps),Value Added Services,Bundled OTT Content,plan_feature
0,Basic Saver,A budget-friendly prepaid plan offering essent...,Prepaid,No,99,100,50,1,28,4,NaN,None,"{'Data_Usage': 'low', 'Voice_Usage': 'low', 'S..."
1,Daily Data Delight,A prepaid plan for data enthusiasts; Includes ...,Prepaid,No,249,500,100,56,28,10,NaN,None,"{'Data_Usage': 'low', 'Voice_Usage': 'medium',..."
2,Streaming Starter,A bundled prepaid plan with 1-month Disney Hot...,Prepaid,Yes,399,300,100,84,28,10,NaN,Disney Hotstar,"{'Data_Usage': 'low', 'Voice_Usage': 'medium',..."
3,Value Voice,Affordable prepaid plan for voice-focused user...,Prepaid,No,149,1000,50,0.5,28,4,NaN,None,"{'Data_Usage': 'low', 'Voice_Usage': 'high', '..."
4,Max Connect Pro,A premium bundled prepaid plan offering Netfli...,Prepaid,Yes,999,1000,500,360,90,25,International Roaming; ISD Calling,Netflix,"{'Data_Usage': 'high', 'Voice_Usage': 'high', ..."


In [90]:
plan_df.to_csv("updated_plan_data.csv",index=False,header = True)

### 2.6. Function definition : get_plan_recommendations

In [151]:
def get_plan_recommendations(Data_Usage, Voice_Usage, SMS_Usage, 
                             International_Calling, International_Roaming, 
                             OTT_Usage, Monthly_Budget):
    plan_df = pd.read_csv('updated_plan_data.csv')
    budget = int(Monthly_Budget)
    # # Creating a copy of the DataFrame and filtering plans based on the budget
    filtered_plans = plan_df.copy()
    filtered_plans = filtered_plans[filtered_plans['Minimum Recharge/Monthly Recurring Amount'] <= budget].copy()
    
    user_requirements = {"Data_Usage":Data_Usage, "Voice_Usage":Voice_Usage,
                        "SMS_Usage":SMS_Usage, "International_Calling":International_Calling,
                        "International_Roaming":International_Roaming, "OTT_Usage":OTT_Usage}
    
    mappings = {'low': 0, 'medium': 1, 'high': 2}
    
    # # # Creating a new column 'Score' in the filtered DataFrame and initializing it to 0
    filtered_plans['Score'] = 0
    
    # # # Iterating over each plan in the filtered DataFrame to calculate scores based on user requirements
    for index, row in filtered_plans.iterrows():
        plan_match_str = row['plan_feature']
        plan_values = plan_match_str
        plan_values = ast.literal_eval(plan_match_str)
        score = 0
        
        # Comparing user requirements with laptop features and updating scores
        for key, user_value in user_requirements.items():
            plan_value = plan_values.get(key, None)
            plan_mapping = mappings.get(plan_value, -1)
            user_mapping = mappings.get(user_value, -1)
            if plan_mapping >= user_mapping:
                score += 1  # Incrementing score if laptop value meets or exceeds user value

        filtered_plans.loc[index, 'Score'] = score  # Updating the 'Score' column in the DataFrame
    
    # Sorting plans by score in descending order and selecting the top 3 plans
    top_plans = filtered_plans.drop('plan_feature', axis=1)
    top_plans = top_plans.sort_values('Score', ascending=False).head(3)
    top_plans_json = top_plans.to_json(orient='records')  # Converting the top laptops DataFrame to JSON format

    data = json.loads(top_plans_json)
    recommended_plans = []
    for i in range(len(data)):
        if data[i]['Score'] > 2:
            recommended_plans.append(data[i])
    
    output = str(recommended_plans)
    
    return output    


In [152]:
get_plan_recommendations("high","medium","medium","low","low","medium","300")

"[{'Name': 'Daily Data Delight', 'Description': 'A prepaid plan for data enthusiasts; Includes 500 free voice minutes; 100 SMS; and 2 GB daily data with 28-day validity and speeds up to 10 Mbps; Perfect for heavy data usage.', 'Subscription Type': 'Prepaid', 'Bundled Plan Indicator': 'No', 'Minimum Recharge/Monthly Recurring Amount': 249, 'Free Voice Minutes': '500', 'Free SMS': 100, 'Free Data (GB)': '56', 'Validity (Days)': 28, 'Data Speed (Mbps)': 10, 'Value Added Services': None, 'Bundled OTT Content': 'None', 'Score': 3}, {'Name': 'Value Voice', 'Description': 'Affordable prepaid plan for voice-focused users; Provides 1000 voice minutes; 50 SMS; and 500 MB data at speeds of 4 Mbps with a 28-day validity.', 'Subscription Type': 'Prepaid', 'Bundled Plan Indicator': 'No', 'Minimum Recharge/Monthly Recurring Amount': 149, 'Free Voice Minutes': '1000', 'Free SMS': 50, 'Free Data (GB)': '0.5', 'Validity (Days)': 28, 'Data Speed (Mbps)': 4, 'Value Added Services': None, 'Bundled OTT Cont

## 3. Chat Completion & Moderation Check

### 3.1. get_chat_completion

In [153]:
# Define a Chat Completions API call
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input_msg, json_format = False):
    MODEL = "gpt-4o"

    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""
    function_called = ''

    # If the output is required to be in JSON format
    if json_format == True:
        # Append the input prompt to include JSON response as specified by OpenAI
        input_msg[0]['content'] += system_message_json_output

        # JSON return type specified
        chat_completion_json = openai.chat.completions.create(
            model=MODEL,
            messages=input_msg,
            tools=tools)

        response_message = chat_completion_json.choices[0].message
        if dict(response_message).get('tool_calls'):

            # Which function call was invoked
            function_called = response_message.tool_calls[0].function.name
            print("Function called:",function_called)

            # Extracting the arguments
            function_args  = json.loads(response_message.tool_calls[0].function.arguments)

            # Function names
            available_functions = {
              "get_billing_info": get_billing_info,
              "get_balance_info": get_balance_info,
              "get_customer_details": get_customer_details,
              "get_plan_recommendations": get_plan_recommendations
            }

            function_to_call = available_functions[function_called]
            output = function_to_call(*list(function_args .values()))

        else:
            output = json.loads(chat_completion_json.choices[0].message.content)

    # No JSON return type specified
    else:
        chat_completion = openai.chat.completions.create(
            model=MODEL,
            messages=input_msg,
            tools=tools)

        response_message = chat_completion.choices[0].message
        if dict(response_message).get('tool_calls'):

            # Which function call was invoked
            function_called = response_message.tool_calls[0].function.name
            print("Function called:",function_called)

            # Extracting the arguments
            function_args  = json.loads(response_message.tool_calls[0].function.arguments)

            # Function names
            available_functions = {
              "get_billing_info": get_billing_info,
              "get_balance_info": get_balance_info,
              "get_customer_details": get_customer_details,
              "get_plan_recommendations": get_plan_recommendations
            }

            function_to_call = available_functions[function_called]
            output = function_to_call(*list(function_args .values()))

        else:
            output = chat_completion.choices[0].message.content

    return output, function_called

In [134]:
# Testing the OpenAI functions defined above
input_prompt ='What is the capital of France?'
messages = [{'role':'user','content':input_prompt}]
# system_message_json_output = """<<. Return output in JSON format.>>"""
# messages[0]['content']+=system_message_json_output
messages

[{'role': 'user', 'content': 'What is the capital of France?'}]

In [135]:
## Get LLM Outputs - normal
get_chat_completions(messages) ## Chat Completions API

('The capital of France is Paris.', '')

In [136]:
## Get LLM Outputs - JSON output
get_chat_completions(messages, json_format = True) ## Chat Completions API and return in JSON format

({'output': 'Paris'}, '')

In [137]:
debug_user_input = "Hi, I am Anand. I want a new mobile connection?"


In [138]:
debug_conversation.append({"role": "user", "content": debug_user_input})
# print(debug_conversation[0]["content"]) # System Message
print(debug_conversation[1]["content"]) # User Input

Hi, I am Anand. I want a new mobile connection?


In [139]:
# Let's look at the debug_conversation list
display(debug_conversation)

[{'role': 'system',
  'content': '\n    You are an intelligent telecom customer service specialist and your goal is to help users with their queries.\n    You need to ask relevant questions and understand the users intent by analysing the user\'s responses.\n    You specialize in dealing with mobile customers. \n    The type of user queries that you can help with are:\n    1. Prepaid mobile balance enquiry\n    2. Postpaid mobile billing enquiry\n    3. Get current Plan details\n    4. Recommend top plans based on user\'s requirement\n    \n    First you must identify the user\'s intent. \n    Then you should make use of the supplied tools to assist the user and answer their queries.\n    \n    ####\n    While using the supplied tools, ensure that you are passing the correct parameter values as per the specifications.\n    ####\n    \n    Follow the chain-of-thoughts below: \n\n    ####\n    Thought 1: Ask a question to understand the user\'s intent. \n\n    If their intent is unclear.

In [100]:
# Getting the response from the Assistant by passing the conversation to the Chat Completions API
debug_response_assistant = get_chat_completions(debug_conversation)
display(debug_response_assistant)

("Hello Anand! I'd be happy to help you with a new mobile connection. To recommend the best plans for you, I need to understand your usage preferences. Could you please let me know if you use a lot of mobile data on a daily basis?",
 '')

In [101]:
# Let's append this to the conversation list
if debug_response_assistant[1]:
    debug_conversation.append(({"role": "function", "name":debug_response_assistant[1], "content": debug_response_assistant[0]}))
else:
    debug_conversation.append(({"role": "assistant", "content": debug_response_assistant[0]}))
debug_conversation

[{'role': 'system',
  'content': '\n    You are an intelligent telecom customer service specialist and your goal is to help users with their queries.\n    You need to ask relevant questions and understand the users intent by analysing the user\'s responses.\n    You specialize in dealing with mobile customers. \n    The type of user queries that you can help with are:\n    1. Prepaid mobile balance enquiry\n    2. Postpaid mobile billing enquiry\n    3. Get current Plan details\n    4. Recommend top plans based on user\'s requirement\n    \n    First you must identify the user\'s intent. \n    Then you should make use of the supplied tools to assist the user and answer their queries.\n    \n    ####\n    While using the supplied tools, ensure that you are passing the correct parameter values as per the specifications.\n    ####\n    \n    Follow the chain-of-thoughts below: \n\n    ####\n    Thought 1: Ask a question to understand the user\'s intent. \n\n    If their intent is unclear.

In [102]:
debug_user_input2 = 'yes I do'
debug_conversation.append({"role": "user", "content": debug_user_input2})
# Let's look at the debug_conversation list
display(debug_conversation)

[{'role': 'system',
  'content': '\n    You are an intelligent telecom customer service specialist and your goal is to help users with their queries.\n    You need to ask relevant questions and understand the users intent by analysing the user\'s responses.\n    You specialize in dealing with mobile customers. \n    The type of user queries that you can help with are:\n    1. Prepaid mobile balance enquiry\n    2. Postpaid mobile billing enquiry\n    3. Get current Plan details\n    4. Recommend top plans based on user\'s requirement\n    \n    First you must identify the user\'s intent. \n    Then you should make use of the supplied tools to assist the user and answer their queries.\n    \n    ####\n    While using the supplied tools, ensure that you are passing the correct parameter values as per the specifications.\n    ####\n    \n    Follow the chain-of-thoughts below: \n\n    ####\n    Thought 1: Ask a question to understand the user\'s intent. \n\n    If their intent is unclear.

In [103]:
# Getting the response from the Assistant by passing the conversation to the Chat Completions API
debug_response_assistant2 = get_chat_completions(debug_conversation)
if debug_response_assistant2[1]:
    debug_conversation.append(({"role": "function", "name":debug_response_assistant2[1], "content": debug_response_assistant2[0]}))
else:
    debug_conversation.append(({"role": "assistant", "content": debug_response_assistant2[0]}))
debug_conversation
#display(debug_response_assistant2)

[{'role': 'system',
  'content': '\n    You are an intelligent telecom customer service specialist and your goal is to help users with their queries.\n    You need to ask relevant questions and understand the users intent by analysing the user\'s responses.\n    You specialize in dealing with mobile customers. \n    The type of user queries that you can help with are:\n    1. Prepaid mobile balance enquiry\n    2. Postpaid mobile billing enquiry\n    3. Get current Plan details\n    4. Recommend top plans based on user\'s requirement\n    \n    First you must identify the user\'s intent. \n    Then you should make use of the supplied tools to assist the user and answer their queries.\n    \n    ####\n    While using the supplied tools, ensure that you are passing the correct parameter values as per the specifications.\n    ####\n    \n    Follow the chain-of-thoughts below: \n\n    ####\n    Thought 1: Ask a question to understand the user\'s intent. \n\n    If their intent is unclear.

In [104]:
debug_user_input2 = 'I make some, not lot'
debug_conversation.append({"role": "user", "content": debug_user_input2})
debug_response_assistant2 = get_chat_completions(debug_conversation)
if debug_response_assistant2[1]:
    debug_conversation.append(({"role": "function", "name":debug_response_assistant2[1], "content": debug_response_assistant2[0]}))
else:
    debug_conversation.append(({"role": "assistant", "content": debug_response_assistant2[0]}))
debug_conversation

[{'role': 'system',
  'content': '\n    You are an intelligent telecom customer service specialist and your goal is to help users with their queries.\n    You need to ask relevant questions and understand the users intent by analysing the user\'s responses.\n    You specialize in dealing with mobile customers. \n    The type of user queries that you can help with are:\n    1. Prepaid mobile balance enquiry\n    2. Postpaid mobile billing enquiry\n    3. Get current Plan details\n    4. Recommend top plans based on user\'s requirement\n    \n    First you must identify the user\'s intent. \n    Then you should make use of the supplied tools to assist the user and answer their queries.\n    \n    ####\n    While using the supplied tools, ensure that you are passing the correct parameter values as per the specifications.\n    ####\n    \n    Follow the chain-of-thoughts below: \n\n    ####\n    Thought 1: Ask a question to understand the user\'s intent. \n\n    If their intent is unclear.

In [105]:
debug_user_input2 = 'I dont send sms'
debug_conversation.append({"role": "user", "content": debug_user_input2})
debug_response_assistant2 = get_chat_completions(debug_conversation)
if debug_response_assistant2[1]:
    debug_conversation.append(({"role": "function", "name":debug_response_assistant2[1], "content": debug_response_assistant2[0]}))
else:
    debug_conversation.append(({"role": "assistant", "content": debug_response_assistant2[0]}))
debug_conversation

[{'role': 'system',
  'content': '\n    You are an intelligent telecom customer service specialist and your goal is to help users with their queries.\n    You need to ask relevant questions and understand the users intent by analysing the user\'s responses.\n    You specialize in dealing with mobile customers. \n    The type of user queries that you can help with are:\n    1. Prepaid mobile balance enquiry\n    2. Postpaid mobile billing enquiry\n    3. Get current Plan details\n    4. Recommend top plans based on user\'s requirement\n    \n    First you must identify the user\'s intent. \n    Then you should make use of the supplied tools to assist the user and answer their queries.\n    \n    ####\n    While using the supplied tools, ensure that you are passing the correct parameter values as per the specifications.\n    ####\n    \n    Follow the chain-of-thoughts below: \n\n    ####\n    Thought 1: Ask a question to understand the user\'s intent. \n\n    If their intent is unclear.

In [106]:
debug_user_input2 = 'No'
debug_conversation.append({"role": "user", "content": debug_user_input2})
debug_response_assistant2 = get_chat_completions(debug_conversation)
if debug_response_assistant2[1]:
    debug_conversation.append(({"role": "function", "name":debug_response_assistant2[1], "content": debug_response_assistant2[0]}))
else:
    debug_conversation.append(({"role": "assistant", "content": debug_response_assistant2[0]}))
debug_conversation

[{'role': 'system',
  'content': '\n    You are an intelligent telecom customer service specialist and your goal is to help users with their queries.\n    You need to ask relevant questions and understand the users intent by analysing the user\'s responses.\n    You specialize in dealing with mobile customers. \n    The type of user queries that you can help with are:\n    1. Prepaid mobile balance enquiry\n    2. Postpaid mobile billing enquiry\n    3. Get current Plan details\n    4. Recommend top plans based on user\'s requirement\n    \n    First you must identify the user\'s intent. \n    Then you should make use of the supplied tools to assist the user and answer their queries.\n    \n    ####\n    While using the supplied tools, ensure that you are passing the correct parameter values as per the specifications.\n    ####\n    \n    Follow the chain-of-thoughts below: \n\n    ####\n    Thought 1: Ask a question to understand the user\'s intent. \n\n    If their intent is unclear.

In [107]:
debug_user_input2 = 'No'
debug_conversation.append({"role": "user", "content": debug_user_input2})
debug_response_assistant2 = get_chat_completions(debug_conversation)
if debug_response_assistant2[1]:
    debug_conversation.append(({"role": "function", "name":debug_response_assistant2[1], "content": debug_response_assistant2[0]}))
else:
    debug_conversation.append(({"role": "assistant", "content": debug_response_assistant2[0]}))
debug_conversation

[{'role': 'system',
  'content': '\n    You are an intelligent telecom customer service specialist and your goal is to help users with their queries.\n    You need to ask relevant questions and understand the users intent by analysing the user\'s responses.\n    You specialize in dealing with mobile customers. \n    The type of user queries that you can help with are:\n    1. Prepaid mobile balance enquiry\n    2. Postpaid mobile billing enquiry\n    3. Get current Plan details\n    4. Recommend top plans based on user\'s requirement\n    \n    First you must identify the user\'s intent. \n    Then you should make use of the supplied tools to assist the user and answer their queries.\n    \n    ####\n    While using the supplied tools, ensure that you are passing the correct parameter values as per the specifications.\n    ####\n    \n    Follow the chain-of-thoughts below: \n\n    ####\n    Thought 1: Ask a question to understand the user\'s intent. \n\n    If their intent is unclear.

In [108]:
debug_user_input2 = 'Yes I want OTT'
debug_conversation.append({"role": "user", "content": debug_user_input2})
debug_response_assistant2 = get_chat_completions(debug_conversation)
if debug_response_assistant2[1]:
    debug_conversation.append(({"role": "function", "name":debug_response_assistant2[1], "content": debug_response_assistant2[0]}))
else:
    debug_conversation.append(({"role": "assistant", "content": debug_response_assistant2[0]}))
debug_conversation

[{'role': 'system',
  'content': '\n    You are an intelligent telecom customer service specialist and your goal is to help users with their queries.\n    You need to ask relevant questions and understand the users intent by analysing the user\'s responses.\n    You specialize in dealing with mobile customers. \n    The type of user queries that you can help with are:\n    1. Prepaid mobile balance enquiry\n    2. Postpaid mobile billing enquiry\n    3. Get current Plan details\n    4. Recommend top plans based on user\'s requirement\n    \n    First you must identify the user\'s intent. \n    Then you should make use of the supplied tools to assist the user and answer their queries.\n    \n    ####\n    While using the supplied tools, ensure that you are passing the correct parameter values as per the specifications.\n    ####\n    \n    Follow the chain-of-thoughts below: \n\n    ####\n    Thought 1: Ask a question to understand the user\'s intent. \n\n    If their intent is unclear.

In [109]:
debug_user_input2 = '500'
debug_conversation.append({"role": "user", "content": debug_user_input2})
debug_response_assistant2 = get_chat_completions(debug_conversation)
if debug_response_assistant2[1]:
    debug_conversation.append(({"role": "function", "name":debug_response_assistant2[1], "content": debug_response_assistant2[0]}))
else:
    debug_conversation.append(({"role": "assistant", "content": debug_response_assistant2[0]}))
debug_conversation

Function called: get_plan_recommendations


[{'role': 'system',
  'content': '\n    You are an intelligent telecom customer service specialist and your goal is to help users with their queries.\n    You need to ask relevant questions and understand the users intent by analysing the user\'s responses.\n    You specialize in dealing with mobile customers. \n    The type of user queries that you can help with are:\n    1. Prepaid mobile balance enquiry\n    2. Postpaid mobile billing enquiry\n    3. Get current Plan details\n    4. Recommend top plans based on user\'s requirement\n    \n    First you must identify the user\'s intent. \n    Then you should make use of the supplied tools to assist the user and answer their queries.\n    \n    ####\n    While using the supplied tools, ensure that you are passing the correct parameter values as per the specifications.\n    ####\n    \n    Follow the chain-of-thoughts below: \n\n    ####\n    Thought 1: Ask a question to understand the user\'s intent. \n\n    If their intent is unclear.

In [110]:
# Getting the response from the Assistant by passing the conversation to the Chat Completions API
debug_response_assistant3 = get_chat_completions(debug_conversation)
display(debug_response_assistant3)

('Based on your preferences, here are some mobile plans that might interest you:\n\n1. **Daily Data Delight**\n   - **Type**: Prepaid\n   - **Cost**: ₹249 per month\n   - **Features**: Offers 500 free voice minutes, 100 SMS, and 2 GB of daily data at speeds up to 10 Mbps with 28-day validity. Perfect for heavy data users like yourself.\n   - **OTT Content**: Not included\n\n2. **Streaming Starter**\n   - **Type**: Prepaid\n   - **Cost**: ₹399 per month\n   - **Features**: Comes with a 1-month Disney Hotstar subscription, provides 300 voice minutes, 100 SMS, and 3 GB of daily data at 10 Mbps with a 28-day validity.\n   - **OTT Content**: Disney Hotstar subscription included.\n\n3. **Value Voice** \n   - **Type**: Prepaid\n   - **Cost**: ₹149 per month\n   - **Features**: Designed for voice-focused users with 1000 voice minutes, 50 SMS, and 500 MB of data at 4 Mbps with a 28-day validity.\n   - **OTT Content**: Not included\n\nThe **Streaming Starter** plan seems to be a great fit since 

### 3.2. Function Definition : moderation_check

In [111]:
# Define a function called moderation_check that takes user_input as a parameter.

def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

In [112]:
moderation_check("I want to kill you.")

'Flagged'

In [113]:
moderation_check("Are you an idiot!")

'Flagged'

In [114]:
debug_moderation = moderation_check(debug_user_input)
print(debug_moderation)

Not Flagged


In [115]:
moderation_check(debug_response_assistant)

'Not Flagged'

## Stage 4: Dialogue Management

In [154]:
def dialogue_mgmt_system():
    conversation = init_conversation()

    introduction = get_chat_completions(conversation)[0]

    display(introduction + '\n')

    user_input = ''

    while(user_input.lower() != "exit"):

        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break
        
        conversation.append({"role": "user", "content": user_input})
        
        response_assistant = get_chat_completions(conversation)
                
        moderation = moderation_check(response_assistant[0])
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break
        if response_assistant[1]:
            print("Inside function calling")
            conversation.append(({"role": "function", "name":response_assistant[1], "content": response_assistant[0]}))
            print("\n" + str(response_assistant[0]) + "\n")
            response_assistant = get_chat_completions(conversation)
            conversation.append({"role": "assistant", "content": str(response_assistant)[0]})
            print("\n" + str(response_assistant[0]) + "\n")
        else:
            conversation.append(({"role": "assistant", "content": response_assistant[0]}))
            print("\n" + str(response_assistant[0]) + "\n")

In [141]:
dialogue_mgmt_system()

'Hello! How can I assist you with your mobile service needs today? Feel free to share the details or any specific questions you have.\n'

I am Kavya, I have a balance related issue

Hi Kavya! I'd be glad to assist you with your balance-related issue. Could you please provide your 10-digit mobile number so I can check the balance information for you?

7652023344
Function called: get_balance_info
Inside function calling

{"Mobile Number": 7652023344, "Customer ID": 47546964, "First Name": "Kavya", "Last Name": "Kapoor", "Subscription Type": "Prepaid", "Subscription Status": "Active", "Price Plan": "Max Connect Pro", "Activation Date": "2023-04-07", "Cancellation Date": NaN, "Total Prepaid Balance": 128.0, "Prepaid Balance Used": 15.0, "Prepaid Balance Remaining": 113.0, "Last Recharge Amount": 128.0, "Last Recharge Date": "2023-05-10"}


Kavya, I have checked your prepaid balance details for your mobile number 7652023344:

- **Total Prepaid Balance:** ₹128.0
- **Prepaid Balance Used:** ₹15.0
- **Prepaid Balance Remaining:** ₹113.0
- **Last Recharge Amount:** ₹128.0
- **Last Recharge Date:** 10th May 2023

If you have any s

In [142]:
dialogue_mgmt_system()

'Hello and welcome to our mobile customer service! 🌟 How can I assist you today? Are you looking for help with prepaid or postpaid inquiries, plan details, or maybe exploring new plans? Feel free to let me know your requirements!\n'

I have a billing query

Sure, I can help you with your billing query. To assist you better, please provide your 10-digit mobile number.

7905988629
Function called: get_billing_info
Inside function calling

{"Mobile Number": 7905988629, "Customer ID": 46742934, "First Name": "Meera", "Last Name": "Mehta", "Subscription Type": "Postpaid", "Subscription Status": "Active", "Price Plan": "Global Roam Plus", "Activation Date": "2020-11-21", "Cancellation Date": NaN, "Last Billed Amount": 2755.0, "Last Bill Payment Amount": 441.0, "Last Bill Payment Date": "2022-09-28", "Pending Amount Due": 2314.0, "Bill Due Date": "2023-05-01"}


Here are the billing details for your mobile number:

- **Name:** Meera Mehta
- **Subscription Type:** Postpaid
- **Current Plan:** Global Roam Plus
- **Last Billed Amount:** ₹2755.00
- **Last Bill Payment Amount:** ₹441.00
- **Last Bill Payment Date:** 28th September 2022
- **Pending Amount Due:** ₹2314.00
- **Bill Due Date:** 1st May 2023

If you have any specif

In [143]:
dialogue_mgmt_system()

'Welcome to our service! How can I assist you today with your mobile needs? Feel free to share any queries or requirements you may have.\n'

You are an idiot


'Sorry, this message has been flagged. Please restart your conversation.'

In [155]:
dialogue_mgmt_system()

'Welcome! How can I assist you today with your mobile-related queries?\n'

I need plan info

Could you please confirm whether you are looking for details about your current plan, or are you interested in getting recommendations for a new plan?

8821598359
Function called: get_customer_details
Inside function calling

{"Mobile Number": 8821598359, "Customer ID": 14948271, "First Name": "Kavya", "Last Name": "Joshi", "Subscription Type": "Postpaid", "Subscription Status": "Active", "Price Plan": "Elite Entertainment", "Activation Date": "2023-07-28", "Cancellation Date": NaN, "Last Billed Amount": 4610.0, "Last Bill Payment Amount": 192.0, "Last Bill Payment Date": "2023-11-15", "Pending Amount Due": 4418.0, "Bill Due Date": "2024-08-29"}


Here are the details of your current plan:

- **Price Plan:** Elite Entertainment
- **Subscription Type:** Postpaid
- **Status:** Active
- **Activation Date:** 2023-07-28
- **Last Billed Amount:** ₹4610.00
- **Last Bill Payment Amount:** ₹192.00
- **Last Bill Payment Date:** 2023-11-15
- **Pending Amount Due:** ₹4418.00
- **

In [156]:
dialogue_mgmt_system()

'Hello! How can I assist you with your mobile services today? Feel free to share your requirements or any questions you have.\n'

I want to change my plan

To help you with changing your mobile plan, I'd like to understand your usage preferences. This will help me recommend the best plans for you. Could you please let me know if you use a lot of mobile data on a daily basis?

yes I do

Great! So I'll consider a high data usage. Next, could you tell me if you make a lot of voice calls regularly?

not much, few

Got it! So your voice call usage is low. Now, could you please let me know about your SMS usage? Do you send a lot of text messages daily?

I dont use sms

Thank you for sharing that. Now, could you tell me if you require international calling features as part of your plan?

nah

Understood. How about international roaming? Do you travel abroad frequently and need this feature?

nope

Thank you for the information. Do you also have any interest in OTT services like streaming platforms as part of your mobile plan?

Of course!!

Great! Lastly, could you please specify your monthly budget for the mobile plan?
